In [2]:
__author__ = 'erwich/sikkel'

In [3]:
import getopt
import os, sys
from sys import argv, exit, stderr

from collections import defaultdict, Counter
from operator import attrgetter
from functools import lru_cache #
import re
from shutil import rmtree
from glob import glob
from pprint import pprint
from heapq import heappush, heappop

import numpy as np
import math
import pandas as pd

from tf.app import use
from sly import Lexer, Parser

from scipy.optimize import minimize, linear_sum_assignment
from translate import translate


In [4]:
TAB = '\t'
NL = '\n'
VERSION = 'c'
TYP_CHOICE = {'NP', 'PP', 'PrNP', 'PPrP', 'VP', 'DPrP', 'IPrP'}

In [5]:
def error(*args, **kwargs):
    print(*args, file=stderr, **kwargs)
    exit(1)
    
def Usage():
    stderr.write('usage: mimi.py book_name first_chapter [last_chapter]\n')
    exit(1)
    
A = use('bhsa:local', 
        checkout='local',
        version = VERSION,
        hoist=globals(),
        silent=True
       )
TF.load('g_prs', add=True)

   |     0.00s Dataset without structure sections in otext:no structure functions in the T-API


In [6]:
class Mention:
    def __init__(self, text='', start=0, end=0, node_tuple=tuple(), 
                 person='', gender='', number='', issuffix = False, rpt ='', function=''):
        
        self.text = text               # Lexical information of the mention
        self.start = start             # Start of the word position in the txt file
        self.end = end                 # End of the position in the txt file
        self.node_tuple = node_tuple   # Text-Fabric nodes of the mention(s)
        self.person = person           # Grammatic person of word object 
        self.gender = gender           # Grammatic gender of word object 
        self.number = number           # Grammatic number of word object 
        self.issuffix = issuffix       # Boolean for existence of suffix on word
        self.rpt = rpt                 # Reconstructed phrase type
        self.function = function       # Function of mention(s) within phrase object (e.g. object, subject)
        self.pargr = ''                # Paragraph number of the mention within the clause_atom object
        self.txttype = ''              # Text type of the mention within the clause object: ?, N, D, Q
        self.name = ''                 # Identifier of the mention, e.g. T32
        self.note = ''                 # AnnotatorNotes generated by MakeMentions()
        self.file = ''                 # File object identifier  
        self.corefclass = set()        # Coreference class to which the mention belongs
        self.who = ''                  # Identification of a class as entity
        
    def __str__(self):
        return self.name
    
    def __repr__(self):
        #return self.name + ' ' + self.text
        return self.text
    
    def __gt__(self, other): 
        return self.node_tuple[0] > other.node_tuple[0]
    
    def __lt__(self, other):
        return self.node_tuple[0] < other.node_tuple[0]
        
class Token:
    '''
    Token object for Sly's lexer. 
    - type: the Token type
    - mention: Mention object
    - node: node number of the word. 
    For suffixes the node of the word it was attached to is used. 
    '''
    def __init__(self, type='', mention='', node=0):
        self.type = type
        self.value = mention
        self.node = node
        
class Statistics:
    '''
    Contains descriptive statistics for the two phases of MiMi:
    1. Mention detection: `pa_count' (phrase atom count), `error_count', and `rule_count'
    2. Coreference resolution: `input_corefs' and `out_corefs'. input_corefs is a list of singleton sets.
    output_corefs is a list of unified coreference classes, singletons and unresolved mentions. 
    '''
    
    def __init__(self):
        
        # Mention detection
        self.pa_count = 0 
        self.error_count = 0
        self.discarded = 0 
        self.created = 0
        self.rule_count = Counter()
        self.fc_parsed = FuzzyCounter()
        self.fc_failed = FuzzyCounter()
        
        # Coreference resolution
        self.input_corefs = 0
        self.output_corefs = 0
        self.coref_classes = 0
        
        # Sieve counters
        self.resolve_predicate = 0
        self.resolve_1p_2p_pronouns = 0
        self.resolve_vocative = 0
        self.resolve_apposition = 0
        self.resolve_fronted = 0
        # extra
        self.match_string = 0
        self.resolve_entity = 0
        self.resolve_3p_pronouns = 0
        
    def __iadd__(self, other):
        self.pa_count += other.pa_count
        self.error_count += other.error_count
        self.input_corefs += other.input_corefs
        self.output_corefs += other.output_corefs
        self.coref_classes += other.coref_classes
        self.resolve_predicate += other.resolve_predicate
        self.resolve_1p_2p_pronouns += other.resolve_1p_2p_pronouns
        self.resolve_vocative += other.resolve_vocative
        self.resolve_apposition += other.resolve_apposition
        self.resolve_fronted += other.resolve_fronted
        self.match_string += other.match_string   # extra
        self.resolve_entity += other.resolve_entity           # extra
        self.resolve_3p_pronouns += other.resolve_3p_pronouns # extra
        self.rule_count += other.rule_count
        self.fc_parsed += other.fc_parsed
        self.fc_failed += other.fc_failed
        return self        
        
    def mention_success(self):
        #return 1 - self.error_count / self.pa_count
        return 1 - self.fc_failed.count / self.fc_parsed.count
    
    def mention_failure(self):
        #return self.error_count / self.pa_count
        return self.fc_failed.count / self.fc_parsed.count
    
    def coref_success(self):
        return (self.input_corefs - self.output_corefs) / self.input_corefs
    
    def coref_unresolved(self):
        return self.output_corefs / self.input_corefs

class FuzzyCounter:
    
    def __init__(self):
        self.count = 0
        self.bonus = 0
        self.malus = 0
    
    def __iadd__(self, other):
        self.count += other.count
        self.bonus += other.bonus
        self.malus += other.malus
        return self
    
    def inc(self):
        self.count += 1
    
    def total(self):
        return self.count + self.bonus - self.malus
    
    def __repr__(self):
        return f'{self.count}+{self.bonus}-{self.malus}'
    
class MyLexer(Lexer):
    '''
    From sly. 
    '''
    
    tokens = {ADJV_A,
             ADJV_C,
             ADJV_E,
             ADVB,
             ART,
             CONJ,
             CONJ_P,
             EOA,
             INRG,
             INTJ,
             NEGA,
             NMPR,
             PRDE,
             PREP,
             PRIN,
             PRPS,
             PRS,
             SUBS_A,
             SUBS_C,
             SUBS_E,
             SUBS_P,
             VERB 
             }
    
    # Define a rule so we can track line numbers
    @_(r'\n+')
    def ignore_newline(self, t):
        self.lineno += len(t.value)
    
    def tokenize(self, my_tokens):
        i = 0
        while i < len(my_tokens):
            yield my_tokens[i]
            i += 1
    
    # Error handling rule
    def error(self, t):
        print("Illegal character '%s'" % t.value[0])
        self.index += 1    

In [7]:
class MyParser(Parser):
    
    def __init__(self, statistics):
        
        super().__init__()
        self.Chapter_Mentions = []
        self.statistics = statistics
        self.created = 0 # Reset per phrase atom. Count extra mention(s) per phrase atom
        self.discarded = 0 # Reset per phrase atom. Count phrase atom
        self.appended = 0 # Counts per phrase atom each time a mention is appended to Chapter_Mentions list
    
    debugfile = 'parser.out'
    tokens = MyLexer.tokens
    
    def append(self, mention):
        self.Chapter_Mentions.append(mention)
        self.appended += 1
    
    def error(self, p):
        '''
        Writes syntax errors for tokens that are not parseable 
        for some reason to a file `mention_errors`. 
        '''
 
        if p:
            mention_errors.write(f'{p.type}, {p.value.text}, start={p.value.start}, node={p.node}\n')
        else:  
            mention_errors.write('Syntax error at EOF\n')
            print('Syntax error at EOF')
        self.statistics.error_count += 1
        
    def fuzzycount(self, fc):
        fc.inc()
        fc.malus += self.discarded
        fc.bonus += self.created
        self.discarded = 0
        self.created = 0
        self.appended = 0
    
    # GRAMMAR RULES FOR BIBLICAL HEBREW PHRASE ATOMS # 
    
    @_('term_atom', 'text term_atom')
    def text(self, p):
        self.statistics.rule_count[self.production.number] += 1
        return self.Chapter_Mentions
    
    @_('phrase_atom EOA')
    def term_atom(self, p):
        self.statistics.rule_count[self.production.number] += 1
        self.fuzzycount(self.statistics.fc_parsed)
        
        # debugging 
        n = self.statistics.rule_count[self.production.number]
        e = self.statistics.error_count
        l = len(self.Chapter_Mentions)
        d = self.statistics.discarded
        c = self.statistics.created
        
        if len(self.Chapter_Mentions) != self.statistics.fc_parsed.total() + self.statistics.fc_failed.total():
            print(self.statistics.fc_parsed, self.statistics.fc_failed)
            print(n, e, l, d, c, f'Het is misgegaan bij {p[0].node_tuple}, {p[0].text}, {p[0].rpt}')
            assert(False)
        return p[0]
    
    @_('error EOA')
    def term_atom(self, p):
        self.statistics.rule_count[self.production.number] += 1
        #print(p[1].text, p[1].start, p[1].end)
        if self.appended == 0:
            self.discarded = 1
        #else:
            # print last sucessful mention in self.Chapter_Mentions[-1]
        #    print(p[0].text, p[0].node_tuple, p[0].rpt)
        self.fuzzycount(self.statistics.fc_failed)
    
    @_('interrogative_phrase', 'adverbial_phrase', 'complex_prepositional_phrase', 
       'verbal_phrase', 'subject_suffix')
    def phrase_atom(self, p):
        self.statistics.rule_count[self.production.number] += 1
        #print('Mention', p[0].text, p[0].start, p[0].end)
        return p[0]
    
    @_('complex_noun_phrase', 'complex_adjective_phrase')
    def phrase_atom(self, p):
        self.statistics.rule_count[self.production.number] += 1
        self.append(p[0])
        return p[0]
    
    @_('normal_VP', 'prepositional_VP')
    def verbal_phrase(self, p):
        self.statistics.rule_count[self.production.number] += 1
        return p[0]
    
    @_('PREP verbal_phrase')
    def prepositional_VP(self, p):
        self.statistics.rule_count[self.production.number] += 1
        return p[1]
    
    @_('VERB')
    def normal_VP(self, p):
        self.statistics.rule_count[self.production.number] += 1
        self.append(p[0])
        return p[0] 
    
    @_('VERB PRS')
    def normal_VP(self, p):
        self.statistics.rule_count[self.production.number] += 1
        self.append(p[0])
        return p[0]
    
    @_('noun_phrase')
    def complex_noun_phrase(self, p):
        self.statistics.rule_count[self.production.number] += 1
        return p[0]
    
    @_('complex_noun_phrase noun_phrase')
    def complex_noun_phrase(self, p):
        self.statistics.rule_count[self.production.number] += 1
        return CombineMentions(p[0], ' ', p[1])
    
    @_('adjective_phrase')
    def complex_adjective_phrase(self, p):
        self.statistics.rule_count[self.production.number] += 1
        return p[0]
    
    @_('complex_adjective_phrase CONJ adjective_phrase')
    def complex_adjective_phrase(self, p):
        self.statistics.rule_count[self.production.number] += 1
        m = CombineMentions(p[0], ' ' + p[1].text, p[2])
        m.number = 'pl'
        return m
    
    @_('complex_noun_phrase CONJ noun_phrase')
    def complex_noun_phrase(self, p):
        self.statistics.rule_count[self.production.number] += 1
        m = CombineMentions(p[0], ' ' + p[1].text, p[2])
        m.number = 'pl'
        return m
    
    @_('complex_prepositional_phrase CONJ_P prepositional_phrase')
    def complex_prepositional_phrase(self, p):
        self.statistics.rule_count[self.production.number] += 1
        self.created += 1
        #print('created', p[0].text, p[2].text)
        m = CombineMentions(p[0], ' ' + p[1].text, p[2])
        m.number = 'pl'
        return m
    
    @_('prepositional_phrase')
    def complex_prepositional_phrase(self, p):
        self.statistics.rule_count[self.production.number] += 1
        return p[0]
    
    @_('focussed_prepositional_phrase')
    def prepositional_phrase(self, p):
        self.statistics.rule_count[self.production.number] += 1
        return p[0]
    
    @_('ADVB focussed_prepositional_phrase')
    def focussed_prepositional_phrase(self, p):
        self.statistics.rule_count[self.production.number] += 1
        return p[1]
    
    @_('extended_prepositional_phrase')
    def focussed_prepositional_phrase(self, p):
        self.statistics.rule_count[self.production.number] += 1
        return p[0]
    
    @_('PREP extended_prepositional_phrase')
    def extended_prepositional_phrase(self, p):
        self.statistics.rule_count[self.production.number] += 1
        return p[1]
    
    @_('PREP complex_noun_phrase')
    def extended_prepositional_phrase(self, p):
        self.statistics.rule_count[self.production.number] += 1
        self.append(p[1])
        return p[1]
        
    @_('PREP ADVB', 'PREP PRIN', 'PREP INRG')
    def prepositional_phrase(self, p):
        self.statistics.rule_count[self.production.number] += 1
        self.discarded += 1
        #print('discarding', p[0].text, p[1].text)
        sep = '' if p[0].text[-1] == '-' else ' '
        return CombineMentions(p[0], sep, p[1])
    
    @_('ADVB complex_noun_phrase')
    def adverbial_phrase(self, p):
        self.statistics.rule_count[self.production.number] += 1
        self.append(p[1])
        return p[1]
        
    @_('ADJV_C noun_phrase')
    def adjective_phrase(self, p):
        self.statistics.rule_count[self.production.number] += 1
        return p[1]
    
    @_('determined_phrase opt_determination')
    def noun_phrase(self, p):
        self.statistics.rule_count[self.production.number] += 1
        if p[1].text == '':
            return p[0]
        else:
            return CombineMentions(p[0], ' ', p[1])

    @_('undetermined_phrase')
    def noun_phrase(self, p):
        self.statistics.rule_count[self.production.number] += 1
        return p[0]
    
    @_('SUBS_C determined_phrase')
    def determined_phrase(self, p):
        self.statistics.rule_count[self.production.number] += 1
        return CombineMentions(p[0], ' ', p[1])
    
    @_('determined_noun')
    def determined_phrase(self, p):
        self.statistics.rule_count[self.production.number] += 1
        return p[0]
    
    @_('ART absolute_noun')
    def determined_noun(self, p):
        self.statistics.rule_count[self.production.number] += 1
        m = CombineMentions(p[0], '', p[1])
        # correct rpt for NP's starting with an article
        m.rpt = p[1].rpt
        return m 
    
    @_('SUBS_A', 'NMPR')
    def absolute_noun(self, p): 
        self.statistics.rule_count[self.production.number] += 1
        return p[0]
    
    @_('PRS', 'NMPR', 'PRPS', 'PRDE')
    def determined_noun(self, p):
        self.statistics.rule_count[self.production.number] += 1
        return p[0]
        
    #for JHWH >DNJ, JHWH <LJWN etc.
    @_('NMPR ADJV_A')
    def determined_noun(self, p):
        self.statistics.rule_count[self.production.number] += 1
        return CombineMentions(p[0], ' ', p[1])
    
    @_('SUBS_C undetermined_phrase')
    def undetermined_phrase(self, p):
        self.statistics.rule_count[self.production.number] += 1
        return CombineMentions(p[0], ' ', p[1])
    
    @_('undetermined_noun')
    def undetermined_phrase(self, p):
        self.statistics.rule_count[self.production.number] += 1
        return p[0]
        
    @_('SUBS_A', 'SUBS_P')
    def undetermined_noun(self, p):
        self.statistics.rule_count[self.production.number] += 1
        return p[0]
    
    @_('SUBS_A ADJV_A')
    def undetermined_noun(self, p):
        self.statistics.rule_count[self.production.number] += 1
        return CombineMentions(p[0], ' ', p[1])
        
    @_('PRIN')
    def interrogative_phrase(self, p):
        self.statistics.rule_count[self.production.number] += 1
        self.discarded += 1
        #print('discarding', p[0].text)
        return Mention('', 0, 0)
    
    @_('')
    def opt_determination(self, p):
        self.statistics.rule_count[self.production.number] += 1
        return Mention('', 0, 0)
    
    @_('ART PRDE', 'ART ADJV_A', 'ART SUBS_A')
    def opt_determination(self, p):
        self.statistics.rule_count[self.production.number] += 1
        m = CombineMentions(p[0], '', p[1])
        # correct rpt for NP's starting with an article
        m.rpt = p[1].rpt
        return m

    @_('INTJ PRS', 'NEGA PRS', 'INRG PRS')
    def subject_suffix(self, p):
        self.statistics.rule_count[self.production.number] += 1
        self.append(p[1])
        return p[1]

Parser debugging for MyParser written to parser.out


In [8]:
def CombineMentions(m1, sep, m2):
    '''
    Function is called in MyParser. 
    - Combines mentions that have been parsed from phrase atoms that contain complex
    noun phrases or prepositional phrases. 
    - Assigns the person, gender and number of the first mention (`m1') to the whole mention construct. 
    - Assigns the issuffix boolean of m1 to the whole mention construct.
    - Assigns the reconstructed phrase type (`rpt') of m1 to the whole mention construct.
    - Assigns the function of m1 to the whole mention construct.
    All assigned information is important for the coreference resolver. 
    '''
    
    node_tuple = (m1.node_tuple[0], m2.node_tuple[1])
    text = m1.text + sep + m2.text
    m = Mention(text, m1.start, m2.end, node_tuple)
    m.person = m1.person
    m.gender = m1.gender
    m.number = m1.number 
    assert(not m1.issuffix)
    m.issuffix = m1.issuffix
    m.rpt = m1.rpt
    assert(m1.function == m2.function)
    m.function = m1.function
    return m
    
def OpenErrorFile(clustername):
    '''
    Opens error analysis files in txt format in current folder
    for the two phases of MiMi:
    - Mention detection
    - Coreference resolution
    '''
    
    filename_mention_detection = f'mention_errors_{clustername}.out'
    
    md_errors = open(filename_mention_detection, 'w')
    
    return md_errors

def OpenChapterFiles(filename):
    '''
    Opens for each chapter a text file with the Hebrew text 
    and a tsv file for Text-Fabric administration
    in current folder. 
    '''
    
    filename_txt = f'{filename}.txt'
    filename_tsv = f'{filename}.tsv'

    txt_f = open(filename_txt, 'w')
    tsv_f = open(filename_tsv, 'w')
    
    return txt_f, tsv_f

def OpenAnn(filename):
    '''
    Opens an annotation file with extension `ann' for each chapter
    in current folder. 
    '''
    
    filename_ann = f'{filename}.ann'
    ann_f = open(filename_ann, 'w')
    
    return ann_f

def make_index(offset, g_word):
    '''
    Produces the start and index of a transliterated Hebrew word. 
    Return an index tuple. 
    '''
    
    start = offset
    end = offset + len(g_word) - 1
    
    return (start, end)

def replace(g_prs):
    '''
    Strips the feature `g_prs' (inspect with: `F.g_prs.freqList()') of the pointed 
    representation of the pronominal suffix of a word in BHSA transliteration.
    It returns the consonantal text of the suffix in string form. 
    Instead of '+@HEM', 'HM' is returned. 
    '''
    
    char_set = set('+:@.,;AEIOU')
    if g_prs not in {'', '+'}:
        new_prs = 'A' if g_prs == '+A' else ''.join(char for char in g_prs if char not in char_set)
        return new_prs
    
def process_suffix(source_string, suffix, replace_what, replace_with):
    '''
    S.rpartition(sep) -> (head, sep, tail)

    Search for the _sep (separator) in S (string), starting at the end of S, and return
    the part before it, the separator itself, and the part after it.  If the
    separator is not found, return two empty strings and S.
    '''
    if 'a' not in suffix:
        if replace_what:
            head, _sep, tail = source_string.rpartition(replace_what)
        # non realised suffixes
        else: 
            head = source_string
            replace_with = '+'
        return head + replace_with
    else:
        return source_string
       
def process_space(gcons_word, trailer, sep):
    '''
    Processes the space between transliterated Hebrew words for the text files. 
    If the feature trailer contains no space a hyphen is placed between the words. 
    In all other cases a space, specified in `sep' is generated.
    '''
    
    if trailer == '':
        return f'{gcons_word}-'
    else:
        return f'{gcons_word}{sep}'

def emit_word(w, sep):
    '''
    Produces a transliterated Hebrew word (string), processed for spaces or trailers. 
    The suffix (g_prs) is processed by adding a '+' to make it more recognisable.
    '''
    
    gcons_word = F.g_cons.v(w)
    trailer = F.trailer.v(w)
    g_prs = F.g_prs.v(w)
    prs = F.prs.v(w)
    new_prs = replace(g_prs)
    g_word = process_suffix(gcons_word, prs, new_prs, f'+{new_prs}')
    g_word = process_space(g_word, trailer, sep)
    return g_word

def WriteChapter(chapterNode, txt_f, tsv_f, filename):
    '''
    For each chapternode (integer), and thus for each Hebrew Bible chapter, two files are made:
    1. A text file with the transliterated Hebrew text for import into brat. 
    The text files are made more readable by adding '-' between words that do not have a space
    if there is a space it is retained. The suffix is given a '+'. 
    2. The tsv files contain for each word, separated by tabs: 
    start index, end index, word node, text of the word. 
    The `index_dict' is used for retrieving word indices to fill the Token object. 
    '''
    
    index_dict = {}
    offset = 0
    
    def writeP(text, fh):
        fh.write(text)
        return len(text)
    
    offset += writeP(f'{filename}\n', txt_f)

    header = ['start_index', 'end_index', 'word_node', 'word']
    tsv_f.write('{}\n'.format('\t'.join(header))) 
    
    for vn in L.d(chapterNode, 'verse'):
        
        verse = T.sectionFromNode(vn)[2]
        verse_words = L.d(vn, 'word')

        # write transcription and .tsv
        offset += writeP(f'{verse} ', txt_f)
        
        for i in range(len(verse_words)):
            w = verse_words[i] 
            g_word = emit_word(w, ' ')
            begin, end = make_index(offset, g_word)
            index_dict[w] = (begin, end)
            
            tsv_f.write(f'{begin}\t{end}\t{w}\t{g_word}\n')
            
            offset += writeP(f'{g_word}', txt_f)
    
        offset += writeP('\n', txt_f)

    return index_dict

def CloseChapterFiles(f1, f2): 
    '''
    Close all txt and tsv file objects.
    '''
    
    f1.close()
    f2.close()
    
def CloseFile(file):
    '''
    Close file object.
    '''
    
    file.close()

def first_phrase_atom(pa):
    '''
    Check whether phrase atom is the first atom of the phrase.
    '''
    phrase = L.u(pa, 'phrase')[0]
    return L.d(phrase, 'phrase_atom')[0] == pa
    
        
def post_process(my_tokens):
    '''
    The function is called in `MakeTokens()'
    The tokens that are made from a selection of phrase atoms as specified in constant
    `TYP_CHOICE' are postprocessed in three cases:
    1. complex prepositional phrases
    2. substantives that are split from suffixes (`prs')
    3. `EOA' (End Of Atom) tokens followed by a `CONJ' (conjunction) 
    '''
    
    for idx, item in enumerate(my_tokens):
        
        # Change token for complex prepositional phrases: CONJ PREP to CONJ_P PREP   
        if item.type == 'PREP' and my_tokens[idx-1].type == 'CONJ':
            my_tokens[idx-1].type = 'CONJ_P'
        
        # Make special token 'SUBS_P' for 'subs' split from prs 
        elif item.type == 'EOA' and my_tokens[idx-1].type == 'SUBS_C':
             my_tokens[idx-1].type = 'SUBS_P'
        
        # if pattern is EOA CONJ delete CONJ
        elif item.type == 'CONJ' and my_tokens[idx-1].type == 'EOA':
             del my_tokens[idx]
        
    return my_tokens

def test_eoa(my_tokens):
    for idx, item in enumerate(my_tokens):
        if item.type == 'EOA' and my_tokens[idx-1].type == 'EOA':
            print('DEZE: ', my_tokens[idx-2].mention.text, my_tokens[idx-2].mention.node_tuple)
        print(item.value.text, item.value.node_tuple)

'''
PTT is called in `MakeTokens()'
phrase translation table. 
It translates phrase dependent part of speech 
to a corresponding phrase type. 
'''

PTT = {'subs': 'NP',
    'prep': 'PP',
    'verb': 'VP',
    'conj': 'CP',
    'nmpr': 'PrNP',
    'art': 'Art',
    'nega': 'NegP',
    'advb': 'AdvP',
    'adjv': 'AdjP',
    'prps': 'PPrP',
    'prde': 'DPrP',
    'intj': 'InjP',
    'inrg': 'InrP',
    'prin': 'InrP'
}

def split_prs(source_string, replace_what, replace_with):
    '''
    S.rpartition(sep) -> (head, sep, tail)
    '''
    
    if replace_what:
        head, _sep, tail = source_string.rpartition(replace_what)
    else: 
        head = source_string
        replace_with = '+'
    return head, replace_with

def print_tokens(token_list, path):
    
    f = open(path, 'a')
    for t in token_list:
        print(t.value.text, t.type, t.value.node_tuple, file=f)
    f.close()

def MakeTokens(chn, stats, index_dict):
    '''
    Retrieves all relevant phrase atom types, specified in `TYP_CHOICE' 
    that are needed for mention detection. The relevant information is stored 
    in `Mention' and added to `Token'. The tokens are appended to the list `MyTokens'. 
    
    MakeTokens() first makes tokens of words that do not have a suffix. 
    If the pdp is not a substantive or adjective the pdp and status (pdp_status) is used for the token type. 
    
    The words that have a suffix are split into a word and suffix part. The token for the word is pdp,
    unless the word pdp is substantive or adjective, then the token is pdp_p. 
    Infinitives with predicate subject are treated as a finite verb, and are thus not split. 
    
    After each phrase atom an EOA, End Of Atom, token, is made. 
    This is also the case for finite verbs and suffixes (not infc + PreS). 
    
    `pa_count' counts the number of processed phrase atoms, this is important for keeping track of
    how succesful the mention detection is. 
    '''
    
    MyTokens = []
    pos = ''
    prs = ''
    gcons_word = ''
    
    for pa in L.d(chn, 'phrase_atom'):
        langs = set(F.language.v(w) for w in L.d(pa, 'word'))
        # check if language is Hebrew, and not Aramaic
        if langs.issubset({'Hebrew'}):
            pa_typ = F.typ.v(pa)
            function = F.function.v(L.u(pa, 'phrase')[0])
            # select preselected phrase atom types and phrase functions with subject suffix 
            if pa_typ in TYP_CHOICE or (function.endswith('S') and first_phrase_atom(pa)):
                pa_words = L.d(pa, 'word')
                pa_text = T.text(pa, fmt='text-trans-plain')
                pa_word = tuple(word for word in pa_words)
                for word in pa_word:
                    pdp = F.pdp.v(word)
                    status = F.st.v(word)
                    #function = F.function.v(L.u(word, 'phrase')[0])
                    vt = F.vt.v(word)
                    gcons_word = F.g_cons.v(word)
                    trailer = F.trailer.v(word)
                    g_prs = F.g_prs.v(word)
                    new_prs = replace(g_prs)
                    ps = F.ps.v(word) # person
                    gn = F.gn.v(word) # gender
                    nu = F.nu.v(word) # number 
                    prs_ps = F.prs_ps.v(word) # person suffix
                    prs_gn = F.prs_gn.v(word) # gender suffix 
                    prs_nu = F.prs_nu.v(word) # number suffix 
                    space_word = process_space(gcons_word, trailer, '')

                    if 'a' in F.prs.v(word):
                        if pdp not in {'subs', 'adjv'}:
                            pos = f'{pdp}'.upper()
                        else:
                            pos = f'{pdp}_{status}'.upper()

                        begin, end = index_dict[word]
                        M = Mention(space_word, begin, end, (word, word), ps, gn, nu, False, PTT[pdp], function)
                        MyTokens.append(Token(pos, M, word))

                    else:
                        prs = f'prs'.upper()
                        g_word, plus_prs = split_prs(gcons_word, new_prs, f'+{new_prs}')

                        # mention index administration for brat files based on input text files 
                        begin_word, end_prs = index_dict[word]
                        begin_prs = end_word = begin_word + len(g_word)

                        # every word with a status is tokenised seperately, except infc with PreS
                        # only subs and adjv in construct when prs follows
                        pos = f'{pdp}_p'.upper() if pdp in {'subs', 'adjv'} else f'{pdp}'.upper()
                        M = Mention(g_word, begin_word, end_word, (word, word), ps, gn, nu, False, PTT[pdp], function)
                        MyTokens.append(Token(pos, M, word))

                        # filter out infinitives with predicate suffix
                        if pdp in {'verb', 'subs'} and function != 'PreS':
                            MyTokens.append(Token('EOA', Mention('*'), pa))
                            stats.pa_count += 1
                        M = Mention(plus_prs, begin_prs, end_prs, (word, word), prs_ps, prs_gn, prs_nu, True, 'PPrP', function)
                        MyTokens.append(Token(prs, M, word))

                        # add end of atom token after suffix if atom boundary is not reached
                        if word != pa_word[-1]:
                            MyTokens.append(Token('EOA', Mention('*'), pa))
                            stats.pa_count += 1

                MyTokens.append(Token('EOA', Mention('*'), pa))
                stats.pa_count += 1
    
    # post processing 
    MyTokens = post_process(MyTokens)
    #print_tokens(MyTokens, 'mytokensNew')
    return MyTokens
    
def MentionParseStats(stats, my_book_name):
    
    # FIX: 
    # For Ezra, Isaiah and Psalms there is a difference between, (e.g. for the Psalms):
    # - pa_success (18553) and stats.fc_parsed.count (18552) of -1
    # - stats.error_count (46) and stats.fc_failed.count (47) of +1
    # For all other books these counts are the same. 
    
    mention_list = []
    pa_success = stats.pa_count - stats.error_count #stats.fc_parsed.count
    success_percent = round(stats.mention_success() *100, 1)
    failure_percent = round(stats.mention_failure() *100, 1)
    mention_corefs = stats.fc_parsed.total() + stats.fc_failed.total()
    
    print('\n',\
        f'Mention Parse Statistics {my_book_name}: \n',\
        f'{stats.pa_count} phrase atoms INPUT \n', \
        f'{stats.fc_parsed.count} phrase atoms SUCCESFULLY parsed \n',\
        f'+{stats.fc_parsed.bonus} extra mentions SUCCESFULLY parsed from phrase atoms \n', \
        f'-{stats.fc_parsed.malus} phrase atoms without mentions \n', \
        f'{stats.fc_failed.count} phrase atom parse ERRORS \n',\
        f'+{stats.fc_failed.bonus} extra mentions SUCCESFULLY parsed from phrase atom errors \n',\
        f'-{stats.fc_failed.malus} phrase atoms without mentions from phrase atom ERRORS \n', \
        f'{mention_corefs} mention coreference input \n', \
        f'{success_percent}% parsing succes \n',\
        f'{failure_percent}% parsing error')
    
    mention_list.append({'book' : my_book_name,
                                'phrase atoms' : stats.pa_count,
                                'pa parsed' : stats.fc_parsed.count,
                                '+m' : stats.fc_parsed.bonus, 
                                '-pa' : stats.fc_parsed.malus, 
                                'pa errors' : stats.fc_failed.count,
                                '+m errors' : stats.fc_failed.bonus, 
                                '-pa errors' : stats.fc_failed.malus, 
                                'mentions': mention_corefs, 
                                '%parsed' : success_percent,
                                '%error' : failure_percent   
        })
    
    mention_stats_df = pd.DataFrame(mention_list)
    mention_stats_df = mention_stats_df[['book', 'phrase atoms', 'pa parsed', '+m', '-pa', 
                                         'pa errors', '+m errors', '-pa errors', 
                                         'mentions', '%parsed', '%error']]
    return mention_stats_df
    
def ProcessText(chn, filename): 
    '''
    Write text and tsv files from chapter integer (nodes).
    The chapter nodes are derived from my_book_name specified in CreateCoref()
    '''
    
    txt_f, tsv_f = OpenChapterFiles(filename)
    index_dict = WriteChapter(chn, txt_f, tsv_f, filename)
    CloseChapterFiles(txt_f, tsv_f)
    return index_dict

def ParseMentions(chn, stats, index_dict):
    '''
    Executes MakeTokens(), MyLexer(), MyParser() 
    and returns a mentions list `Mentions' with mention objects.
    '''
    
    Mentions = []
    MyTokens = MakeTokens(chn, stats, index_dict)
    
    if len(MyTokens): 
        Lexer = MyLexer()
        Parser = MyParser(stats)
        Mentions = Parser.parse(Lexer.tokenize(MyTokens))
    
    return Mentions

def PlaceMentions(mentions, ann_file):
    '''
    Relocates the mentions to the corresponding ann file. 
    '''
    
    i = 0
    for m in mentions:
        i += 1
        m.name = f'T{str(i)}'
        m.file = ann_file 
        ann_file.write(f'{m.name}{TAB}Mention {str(m.start)} {str(m.end)}{TAB}{m.text}{NL}')

def EnrichMentions(mentions):
    '''
    Enrich mentions with: 
    - Text type: F.txt.v(clause)
    - Paragraph number: F.pargr.v(clause_atom)
    '''
    
    for m in mentions:
        for clause in L.u(m.node_tuple[0], 'clause'):
            m.txttype = F.txt.v(clause)
            for clause_atom in L.d(clause, 'clause_atom'):
                m.pargr = F.pargr.v(clause_atom)

def MakeCorefSets(mentions):
    '''
    Initially MiMi processes a coreference list of singletons per text, 
    and merges them if they meet the criteria of the applied coreference sieves. 
    The coreference list of singleton sets is made here. 
    '''
    
    coref_list = []
    for m in mentions:
        m.corefclass = {m}
        coref_list.append(m.corefclass)
    return coref_list

def CheckList(lst):
    '''
    Checks the integrity of the coreference list and prints it. The class feature
    of the mentions in the sets should point to the right set.
    '''
    
    for c in lst:
        for m in c:
            assert(m.corefclass == c)
    pprint(lst)

def Unite(lst, i1, i2):
    '''
    Unite(CorefList, i1, i2) is an operation on the CorefList. 
    The mentions from the class with index i2 are added to 
    those of i1. The class with index i2 is removed after 
    this operation. 
    '''
    
    if i1 != i2:
        lst[i1] |= lst[i2]
        for mention in lst[i2]:
            mention.corefclass = lst[i1]
        lst.pop(i2)
        
def find_clause(m):
    '''
    Iterate over clauses that contain the mention of choice:
    e.g. mentions that have reconstructed phrase type 'Verbal Phrase'. 
    '''
    return L.u(m.node_tuple[0], 'clause')[0]

def mention_from_phrase(phrase, mentions):
    '''
    Return first mention word node(s) 
    contained within the phrase
    '''
    
    words = L.d(phrase, 'word')
    for m in mentions: 
        if m.node_tuple[0] in words:
            return m
    return None

def find_phrase_function(c, pf_set):
    '''
    Finds phrases with function from
    phrase_function_set in clause c. 
    '''
    
    for p in L.d(c, 'phrase'):
        if F.function.v(p) in pf_set:
            return p
    return None 

def suitable_mother(clause):
    '''
    Finds the mother clause of clauses that have a clause type
    that ends with '0' within the same text domain (= feature F.txt.v()). 
    '''
    
    first_clause_atom = L.d(clause, 'clause_atom')[0]
    mother_list = E.mother.f(first_clause_atom) or []
    
    clause_type = F.typ.v(clause)
    if len(mother_list) and clause_type.endswith('0'): 
        mca = mother_list[0]
        mother_clause = L.u(mca, 'clause')[0]
        if F.txt.v(mother_clause) == F.txt.v(clause):
            return mother_clause
    return None

def known(v):
    '''
    Returns values for person, gender, number that are known.
    '''
    return v != 'NA' and v != 'unknown'

def disagreement(v1, v2):
    '''
    Checks for disagreement between returned values of known(). 
    '''
    return known(v1) and known(v2) and (v1 != v2)
    
def do_agree(mother_pred_phrase, daughter_pred_phrase):
    '''
    Checks if the predicate phrase of the mother and daughter 
    agree. The words in the mother and daughter predicate with a specific 
    phrase dependent part of speech ('pdp_choice') need to agree. 
    '''
    
    pdp_choice = {'verb', 'subs', 'nmpr', 'prps', 'prde', 'adjv'}
    mother_word = None 
    daughter_word = None 
    
    for word in L.d(mother_pred_phrase, 'word'):
        pdp = F.pdp.v(word)
        if pdp in pdp_choice:
            mother_word = word
            break
           
    for word in L.d(daughter_pred_phrase, 'word'):
        pdp = F.pdp.v(word)
        if pdp in pdp_choice:
            daughter_word = word
            break
    
    # in a few books there is only a mother word 
    # and no daughter word (returns None), 
    # since the end of the hierarchy has been reached:
    if mother_word and daughter_word:
    
        return not disagreement(F.ps.v(mother_word), F.ps.v(daughter_word)) and \
               not disagreement(F.nu.v(mother_word), F.nu.v(daughter_word)) and \
               not disagreement(F.gn.v(mother_word), F.gn.v(daughter_word))
    
def CheckCoref(corefs, statement):
    n = 0
    for c in corefs:
        gn = ''
        for m in c:
            if m.gender in ['f', 'm']:
                if gn == '':
                    gn = m.gender
                elif gn != m.gender:
                    print(m.node_tuple, m.text, m.person, m.gender, m.number, sep='\t')
                    n+=1
    if n > 0:
        print(statement, n)
                
def ResolvePredicate(mentions, coref_list, stats, wv):
    '''
    MiMi's coreference resolution uses the predicate - subject relation as basis 
    for the rest of the sieves. 
    First mother and daughter clauses with an explicit subject phrase are retrieved. 
    If there is no subject phrase the function walks back to the mother clause,
    checks if mother and daughter agree for person, number and gender, then unites them.
    '''
    
    before = len(coref_list)
    
    for mpred in mentions:
        if mpred.rpt == 'VP':
            daughter_clause = find_clause(mpred)
            subj_phrase = find_phrase_function(daughter_clause, {'Subj'})
            if subj_phrase: 
                msubj = mention_from_phrase(subj_phrase, mentions)
                if msubj:
                    Unite(coref_list, coref_list.index(mpred.corefclass), 
                                 coref_list.index(msubj.corefclass))

            else:
                mother_clause = suitable_mother(daughter_clause)
                daughter_pred_phrase = find_phrase_function(daughter_clause, 
                                                            {'Pred', 'PreC', 'PreO', 'PreS', 'PtcO'})
                if mother_clause:
                    mother_pred_phrase = find_phrase_function(mother_clause, {'Pred', 'PreO', 'PreS'})
                    if mother_pred_phrase:
                        mpredmother = mention_from_phrase(mother_pred_phrase, mentions)
                        if mpredmother and do_agree(mother_pred_phrase, daughter_pred_phrase):
                            Unite(coref_list, coref_list.index(mpred.corefclass), 
                                     coref_list.index(mpredmother.corefclass))
    after = len(coref_list)
    stats.resolve_predicate = before - after
              
        
def mention_from_domain(mp, mentions):
    '''
    Checks if other mentions than mp (= mention person):
    - are in the same paragraph as mp;
    - has a higher node number than mp (meaning it appears later in the text);
    - has the same grammatical person 1 or 2 as mp;
    - if the reconstructed phrase type of mention is not a verbal phrase.
    '''
    
    for m in mentions:
        if \
        (m.pargr == mp.pargr) and \
        (m.node_tuple[0] > mp.node_tuple[0]) and \
        (m.person == mp.person) and \
        (m.rpt != 'VP'):
            return m
    return None

def ResolveFirstSecondPersonPronouns(mentions, coref_list, stats, wv):
    '''
    - Link 1P pronouns, `mp1', within same domain, except finite verbs
    - Link 2P pronouns, `mp2', within same domain, except finite verbs
    
    To do: 
    - 3P: Walk back to first word group for which 3P, g, n are the same)
    - Maybe: if mention in iteration has already been added to coref class:
        - Unite 1P coref with predicate coref class
    '''
    
    before = len(coref_list)
    
    mp1 = None
    mp2 = None
    
    for m in mentions:
        if m.person == 'p1' and m.rpt != 'VP':
            mp1 = m
            mp1_same_domain = mention_from_domain(mp1, mentions)
            if mp1_same_domain:
                
                Unite(coref_list, coref_list.index(mp1.corefclass), 
                      coref_list.index(mp1_same_domain.corefclass))
        
        elif m.person == 'p2' and m.rpt != 'VP':
            mp2 = m
            
            mp2_same_domain = mention_from_domain(mp2, mentions)
            if mp2_same_domain:

                Unite(coref_list, coref_list.index(mp2.corefclass), 
                      coref_list.index(mp2_same_domain.corefclass))
                
    after = len(coref_list)
    stats.resolve_1p_2p_pronouns = before - after
    
def mention_from_vocative_domain(mvoct, mentions):
    for m in mentions:
        if \
        (m.pargr == mvoct.pargr) and \
        (m.txttype == mvoct.txttype) and \
        (m.node_tuple[0] > mvoct.node_tuple[0]) and \
        (m.person == 'p2'):
            return m
    return None

def ResolveVocative(mentions, coref_list, stats, wv):
    ''' 
    If a mention has phrase function `Voct':
        - Add all 2P mentions with same F.txt.v(clause) and F.pargr.v(clause_atom) 
        (= same domain) to vocative coreference class
        - Unite vocative coref class with 2P predicate coref class
    '''
    before = len(coref_list)
    
    for m in mentions:
        if m.function == 'Voct':
            mvoct = m
            mvoct_other = mention_from_vocative_domain(mvoct, mentions)
            if mvoct_other and mvoct.corefclass != mvoct_other.corefclass:
                
                Unite(coref_list, coref_list.index(mvoct.corefclass), 
                      coref_list.index(mvoct_other.corefclass))
                
    after = len(coref_list)
    stats.resolve_vocative = before - after 
    
def find_phrase_atom(m):
    return L.u(m.node_tuple[0], 'phrase_atom')[0]

def find_phrase_atom_relation(phrase_atom, pa_rela):
    rela = F.rela.v(phrase_atom)
    if rela == pa_rela:
        return phrase_atom
    return None

def mention_from_apposition(pa_appo, mentions):
    '''
    Return first mention word node(s) 
    contained within the phrase
    '''
    words = L.d(pa_appo, 'word')
    for m in mentions: 
        if m.node_tuple[0] in words:
            return m
    return None
    
def mentions_from_preceding_phrase_atom(mappo, mentions):
    '''
    Finds the mention(s) that belongs to the 
    apposition relation in the mother phrase atom. 
    '''
    
    for m in mentions:
        for pa in L.u(mappo.node_tuple[0], 'phrase_atom'):
            mother_pa_list = E.mother.f(pa) or []
            mpa = mother_pa_list[0]
            pa_words = L.d(mpa, 'word')
            if m.node_tuple[0] in pa_words:
                return m
    return None
    
def ResolveApposition(mentions, coref_list, stats, wv):
    '''
    Finds the mentions that have an apposition relation. 
    First find the mention with phrase_atom `rela' `appo', 
    then find the preceding mentions that belong to the apposition
    through the mother phrase atom relation. 
    '''
    
    before = len(coref_list)
    
    for m in mentions:
        phrase_atom = find_phrase_atom(m)
        if phrase_atom:
            pa_appo = find_phrase_atom_relation(phrase_atom, 'Appo')
            if pa_appo:
                mention_appo = mention_from_apposition(pa_appo, mentions)
                mappo_mentions = mentions_from_preceding_phrase_atom(mention_appo, mentions)
                if mappo_mentions and mention_appo.corefclass != mappo_mentions.corefclass:
                    
                    Unite(coref_list, coref_list.index(mention_appo.corefclass), 
                      coref_list.index(mappo_mentions.corefclass))
                    
    after = len(coref_list)
    stats.resolve_apposition = before - after 
    
def get_frnt_phrase(mfronted):
    return L.u(mfronted.node_tuple[0], 'phrase')[0]

def is_suffix_in(text_object):
    for word in L.d(text_object, 'word'):
        if F.prs.v(word) not in {'absent', 'n/a'}:
            return True
    return False

def find_resu_clause(clause_fronted):
    
    daughter_list = E.mother.t(clause_fronted) or []
    if len(daughter_list):
        dc = daughter_list[0]
        clause_rela = F.rela.v(dc)
        if clause_rela == 'Resu':
            return dc
    return None

def find_resu_phrase(resu_clause):
    for phrase in L.d(resu_clause, 'phrase'):
        phrase_rela = F.rela.v(phrase)
        if phrase_rela == 'Resu':
            return phrase
    return None

def find_other_phrase(resu_clause, function, typ):
    for phrase in L.d(resu_clause, 'phrase'):
        phrase_function = F.function.v(phrase)
        phrase_type = F.typ.v(phrase)
        
        if phrase_function == function and phrase_type == typ:
            return phrase
        elif phrase_function == function and typ == None:
            return phrase
    return None

def find_suffix_in(text_object, mentions):
    words = L.d(text_object, 'word')
    for m in mentions:
        if m.node_tuple[0] in words and m.issuffix:
            return m
    return None

def find_first_mention_in(frnt_phrase, mentions):
    words = L.d(frnt_phrase, 'word')
    for m in mentions:
        if m.node_tuple[0] == words[0]:
            return m
    return None
    
def find_mention_in(phrase, mentions):
    words = L.d(phrase, 'word')
    for m in mentions:
        if m.node_tuple[0] in words:
            return m
    return None
        
def ResolveFrontedElement(mentions, coref_list, stats, wv):
    '''
    * disclaimer: Resu has not been coded for large parts of the BHSA.
    Heuristics were used. 
    
    - Search for mentions that have phrase function fronted element `frnt'.
    - Retrieve the clause that contains the fronted element. 
    - Retrieve the daughter of the fronted clause: resu clause
    - Check in the resu clause for phrase relation `Resu' 
    - If not there, use heuristics. 
    '''
    
    before = len(coref_list)
    
    for m in mentions:
        if m.function == 'Frnt': 
            frnt_phrase = get_frnt_phrase(m)
            m = find_first_mention_in(frnt_phrase, mentions)
            if m:
                clause_fronted = find_clause(m)
                resu_clause = find_resu_clause(clause_fronted)
                if resu_clause:
                    mfronted = m
                    resu_phrase = find_resu_phrase(resu_clause)

                    if resu_phrase:
                        resu_mention = find_mention_in(resu_phrase, mentions)

                        if resu_mention and not is_suffix_in(resu_phrase):
                            Unite(coref_list, coref_list.index(mfronted.corefclass), 
                                  coref_list.index(resu_mention.corefclass))
                            
                        elif resu_mention and is_suffix_in(resu_phrase):
                            resu_suffix = find_suffix_in(resu_phrase, mentions)
                            
                            Unite(coref_list, coref_list.index(mfronted.corefclass), 
                                  coref_list.index(resu_suffix.corefclass))

                        elif resu_mention and resu_mention.issuffix:
                            
                            Unite(coref_list, coref_list.index(mfronted.corefclass), 
                                  coref_list.index(resu_mention.corefclass))

                    # heuristics for if there is no phrase relation `resu'
                    else:
                        #if (mfronted.rpt == 'PPrP' or mfronted.rpt == 'NP') and not mfronted.issuffix:
                        prec_vp_phrase = find_other_phrase(resu_clause, 'PreC', 'VP')
                        
                        if mfronted.rpt == 'PPrP' and not mfronted.issuffix:
                            mods_phrase = find_other_phrase(resu_clause, 'ModS', None)
                            pres_vp_phrase = find_other_phrase(resu_clause, 'PreS', 'VP')
                            preo_vp_phrase = find_other_phrase(resu_clause, 'PreO', 'VP')
                            subj_phrase = find_other_phrase(resu_clause, 'Subj', None)
                            cmpl_phrase = find_other_phrase(resu_clause, 'Cmpl', None)
                            pred_vp_phrase = find_other_phrase(resu_clause, 'Pred', 'VP')
                            
                            # Subject suffix            
                            # 5 cases Gen 44:14; 1Kgs 12:02, 20:40; Jer 33:01, 2Chr 34:03
                            if mods_phrase: 
                                mods_mention = find_mention_in(mods_phrase, mentions)
                                if mods_mention and mods_mention.issuffix:
                                    
                                    # check for agreement 
                                    if not disagreement(mfronted.person, mods_mention.person) and \
                                        not disagreement(mfronted.number, mods_mention.number) and \
                                        not disagreement(mfronted.gender, mods_mention.gender):
                                        Unite(coref_list, coref_list.index(mfronted.corefclass), 
                                              coref_list.index(mods_mention.corefclass))
                            
                            # Predicate object suffix 
                            # 3 cases EZE 30:18; HOS 08:06; ICHR09:22
                            elif preo_vp_phrase:
                                preo_vp_mention = find_mention_in(preo_vp_phrase, mentions)
                                if preo_vp_mention and preo_vp_mention.issuffix:
                                    
                                    # check for agreement 
                                    if not disagreement(mfronted.person, preo_vp_mention.person) and \
                                        not disagreement(mfronted.number, preo_vp_mention.number) and \
                                        not disagreement(mfronted.gender, preo_vp_mention.gender):
                                        Unite(coref_list, coref_list.index(mfronted.corefclass), 
                                              coref_list.index(preo_vp_mention.corefclass))
                            
                            # Suffix on subject and complement
                            # 3 cases EZE 33:17; QOH 07:26; THR 01:04
                            elif subj_phrase:
                                subj_mention = find_mention_in(subj_phrase, mentions)
                                if subj_mention and subj_mention.issuffix:
                                    
                                    # check for agreement 
                                    if not disagreement(mfronted.person, subj_mention.person) and \
                                        not disagreement(mfronted.number, subj_mention.number) and \
                                        not disagreement(mfronted.gender, subj_mention.gender):
                                        Unite(coref_list, coref_list.index(mfronted.corefclass), 
                                              coref_list.index(subj_mention.corefclass))                   
                            elif cmpl_phrase:
                                cmpl_mention = find_mention_in(cmpl_phrase, mentions)
                                if cmpl_mention and cmpl_mention.issuffix:
                                    
                                    # check for agreement 
                                    if not disagreement(mfronted.person, cmpl_mention.person) and \
                                        not disagreement(mfronted.number, cmpl_mention.number) and \
                                        not disagreement(mfronted.gender, cmpl_mention.gender):
                                        Unite(coref_list, coref_list.index(mfronted.corefclass), 
                                              coref_list.index(cmpl_mention.corefclass))
                            
                            # Predicate and subject 
                            # 3 cases GEN 40,10; ISAM 20:29; ISA 57:06        
                            elif pred_vp_phrase and subj_phrase:
                                pred_vp_mention = find_mention_in(pred_vp_phrase, mentions)
                                subj_mention = find_mention_in(subj_phrase, mentions)
                                if pred_vp_mention:
                                    
                                    Unite(coref_list, coref_list.index(mfronted.corefclass), 
                                      coref_list.index(pred_vp_mention.corefclass))
                                if subj_mention.issuffix:
                                    
                                    Unite(coref_list, coref_list.index(mfronted.corefclass), 
                                      coref_list.index(subj_mention.corefclass))
                            
                            # Only predicate
                            # 4 cases Eze 02:05, 04:12; HAB 01:10 2x       
                            elif pred_vp_phrase:
                                pred_vp_mention = find_mention_in(pred_vp_phrase, mentions)
                                if pred_vp_mention:
                                    
                                    Unite(coref_list, coref_list.index(mfronted.corefclass), 
                                      coref_list.index(pred_vp_mention.corefclass))

    after = len(coref_list)
    stats.resolve_fronted = before - after

In [9]:
def pronoun(w):
    '''
    Boolean function that checks for the existence of a pronoun.
    '''
    return F.pdp.v(w) in {'prps', 'prde', 'prin'}
        
def is_kol(w):
    '''
    Boolean function that chscks if 'KL/' is head of a phrase.
    '''
    return F.lex.v(w) == 'KL/' and F.st.v(w) == 'c'
    
def phrase_head(m):
    '''
    Finds the head of the phrase in which the mention (`m`)
    is contained. The head is found by searching 
    for the first word in the phrase 
    with a nominal ending (`nme`). Of that word 
    the lexeme is returned. 
    '''
    
    phrase = L.u(m.node_tuple[0], 'phrase')[0]
    for word in L.d(phrase, 'word'):
        if m.issuffix:
            return F.prs.v(word)
        elif F.nme.v(word) not in {'n/a', 'absent'} and not is_kol(word) or pronoun(word):
            return F.lex.v(word)
    return None

def match_head(m1, m2):
    '''
    The function phrase_head returns a lexeme or None. 
    If `lex1` (lexeme of `m1`) and `lex2` (lexeme of `m2`) 
    have a value a boolean is returned if 
    lex1 and lex2 match on lexeme. 
    The lexemes cannot be contained in the same phrase. 
    This is checked with `phr1` and `phr1`. 
    '''

    lex1 = phrase_head(m1)
    lex2 = phrase_head(m2)
    phr1 = L.u(m1.node_tuple[0], 'phrase')[0]
    phr2 = L.u(m2.node_tuple[0], 'phrase')[0]
    
    # filter out mentions that are contained within the same phrase
    return phr1 != phr2 and lex1 and lex2 and lex1 == lex2
    
def distance_component(d):
    '''
    Calculates the distance component of words in a text
    of the feature vector match_vector with log. 
    `d`,integer, is the distance in word nodes between words. 
    The 0.4 is a scaling factor has been radomnly chosen, 
    it decreases the log(d) value for readability. 
    '''
    v = 0.4 * math.log(d)
    return v
    
def vector_length(match_vector):
    '''
    Calculates and returns the euclidic length of the list `match_vector`.
    `match_vector` contains a number of components. 
    Each component is a numeric value that represents 
    the amount of agreement between two mentions for a specific feature. 
    `vector_length` is called in `score_match`. 
    '''
    i_sum = 0
    euclidic = 0
    for i in match_vector:
        i_sqrt = i**2
        i_sum += i_sqrt
        euclidic = math.sqrt(i_sum)  
    return euclidic

def score_match(m1, m2, w):
    '''
    Function scores the match between two mentions 
    `m1` and `m2` that are compared for agreement. 
    `w` is a (list) vector with weights
    per feature that is compared. `w` can be adjusted in
    the sieves MatchString(), ResolveEntity(), 
    ResolveThirdPersonPronouns().
    Funtions returns a score which is the result
    of the product of each element in match_vector and `w`. 
    '''
    match_vector = []
    txt_distance = m1.node_tuple[0] - m2.node_tuple[0]
    # distance in words with math.log(d)
    d = distance_component(txt_distance)
    match_vector.append(d)
    match_vector.append(disagreement(m1.gender, m2.gender))
    match_vector.append(disagreement(m1.number, m2.number))
    w = list(w)
    w.insert(0, 1.0) # insert value 1.0 to weigh the distance = [1] + w
    w = tuple(w)
    
    return vector_length(np.multiply(w, match_vector))

In [10]:
def MatchString(mentions, coref_list, stats, wv):
    '''
    Matches lexeme strings (not suffixes) exactly by finding for each 
    singleton mention the best possible candidate 
    mention in a class. The best candidate is found by:
    - walking backwards in the text to the beginning;
    - assigning a match score between the two compared mentions (m1, m2);
    - the score is calculated with a feature weights vector and the length
    of the match vector;
    - all canditate mentions for m1 are stored in a heap;
    - the best mention m2 with the best score is popped from the heap
    and united with m1;
    - after the push, a new heap is made for a new mention singleton.
    '''
    before = len(coref_list) 
    feature_weights = [1.0, 1.0]
    unite = []
    for m in mentions:
        if len(m.corefclass) == 1 and m.rpt != 'PrNP' and not m.issuffix:
            heap = []
            for clss in coref_list:
                sorted_clss = sorted(clss, key=attrgetter('node_tuple'))
                i = 0
                while i < len(sorted_clss) and m > sorted_clss[i]:
                    m2 = sorted_clss[i]
                    if not m2.issuffix and match_head(m, m2):
                        score = score_match(m, m2, feature_weights)
                        heappush(heap, (score, m2)) #list of tuples
                    i += 1
            
            if len(heap) > 0:
                best_match = heappop(heap)
                Unite(coref_list, coref_list.index(m.corefclass), 
                      coref_list.index(best_match[1].corefclass))
                #print('united strings', (m.text, m.node_tuple), (best_match[1].text, best_match[1].node_tuple), '\n')
                unite.append((m.text, best_match[1].text))
    #print('unite_string_list', len(unite), unite, '\n')                   
    
    after = len(coref_list)
    stats.match_string = before - after

In [11]:
def ResolveEntity(mentions, coref_list, stats, wv):
    '''
    Resolves named entities as coded in the BSHA data by finding for each 
    singleton mention the best possible candidate 
    mention in a class. The best candidate is found by:
    - walking backwards in the text to the beginning;
    - assigning a match score between the two compared mentions (m1, m2);
    - the score is calculated with a feature weights vector and the length
    of the match vector;
    - all canditate mentions for m1 are stored in a heap;
    - the best mention m2 with the best score is pushed from the heap;
    and united with m1;
    - after the push, a new heap is made for a new mention singleton.
    '''
    before = len(coref_list)
    
    feature_weights = [0.0, 0.0]
    unite = []
    for m in mentions:
        if len(m.corefclass) == 1 and m.rpt == 'PrNP':
            heap = []
            for clss in coref_list:
                sorted_clss = sorted(clss, key=attrgetter('node_tuple'))
                i = 0
                while i < len(sorted_clss) and m > sorted_clss[i]:
                    m2 = sorted_clss[i]
                    if m2.rpt == 'PrNP' and match_head(m, m2):
                        score = score_match(m, m2, feature_weights)
                        heappush(heap, (score, m2)) #list of tuples
                    i += 1
                    
            if len(heap) > 0:
                best_match = heappop(heap)
                Unite(coref_list, coref_list.index(m.corefclass), 
                      coref_list.index(best_match[1].corefclass))
                #print('united entities', (m.text, m.node_tuple), (best_match[1].text, best_match[1].node_tuple), '\n')
                unite.append((m.text, best_match[1].text))
    #print('unite_entity_list', len(unite), unite, '\n')    
          
    after = len(coref_list)
    stats.resolve_entity = before - after

In [12]:
def score_match_3p(m1, m2, w):
    '''
    Function scores the match between two mentions 
    `m1` and `m2` that are compared for agreement. 
    `w` is a (list) vector with weights
    per feature that is compared. `w` can be adjusted in
    the sieve ResolveThirdPersonPronouns. 
    Funtion returns a score which is the result
    of the product of each element in match_vector and `w`. 
    '''
    match_vector = []
    txt_distance = m1.node_tuple[0] - m2.node_tuple[0]
    # distance in words with math.log(d)
    d = distance_component(txt_distance)
    match_vector.append(d)
    
    match_vector.append(disagreement(m1.gender, m2.gender))
    match_vector.append(disagreement(m1.number, m2.number))
    
    w = list(w)
    w.insert(0, 1.0) # insert value 1.0 to weigh the distance
    w = tuple(w)
    
    return vector_length(np.multiply(w, match_vector))

def ResolveThirdPersonPronouns(mentions, coref_list, stats, wv):
    '''
    Resolves third person pronouns by finding for each 
    singleton 3p pronoun mention the best possible candidate 
    mention in a class. The best candidate is found by:
    - walking backwards in the text to the beginning;
    - assigning a match score between the two compared mentions (m1, m2);
    - the score is calculated with a feature weights vector and the length
    of the match vector;
    - all canditate mentions for m1 are stored in a heap;
    - the best mention m2 with the best score is pushed from the heap;
    and united with m1;
    - after the push, a new heap is made for a new mention singleton.
    '''
    before = len(coref_list)
    feature_weights = [2.0, 3.6] #[0.002403569522293084, 0.5033652397875054]
    unite = []
    for m in mentions:
        if len(m.corefclass) == 1 and m.person == 'p3': #issuffix
            heap = []
            for clss in coref_list:
                sorted_clss = sorted(clss, key=attrgetter('node_tuple'))
                i = 0
                while i < len(sorted_clss) and m > sorted_clss[i]:
                    m2 = sorted_clss[i]
                    if m2.person == 'p3':
                        score = score_match(m, m2, feature_weights)
                        heappush(heap, (score, m2)) #list of tuples
                    i += 1
            
            if len(heap) > 0:
                best_match = heappop(heap)
                Unite(coref_list, coref_list.index(m.corefclass), 
                      coref_list.index(best_match[1].corefclass))
                #print('united 3p pronouns', (m.text, m.node_tuple), (best_match[1].text, best_match[1].node_tuple), '\n')
                unite.append((m.text, best_match[1].text))
    #print('unite_3p_list', len(unite), unite, '\n')
    
    after = len(coref_list)
    stats.resolve_3p_pronouns = before - after

In [13]:
def MakeSieveList():
    
    '''
    Appends sieve functions to sieve_list.
    The function ExecuteSieves(sieve_list, mentions, corefs) 
    executes the sieves. 
    '''
    
    sieve_list = []
    sieve_list.append(ResolveFirstSecondPersonPronouns)
    sieve_list.append(ResolveVocative)
    sieve_list.append(MatchString)
    sieve_list.append(ResolveEntity)
    sieve_list.append(ResolveApposition)
    sieve_list.append(ResolveFrontedElement)
    sieve_list.append(ResolvePredicate)
    sieve_list.append(ResolveThirdPersonPronouns)
    
    return sieve_list

def ExecuteSieves(sieve_list, mentions, corefs, stats, wv):
    '''
    Executes sieves in sieve_list:
    ResolvePredicate(mentions, corefs)
    ResolveFirstSecondPersonPronouns(mentions, corefs)
    ResolveVocative(mentions, corefs)
    ResolveApposition(mentions, corefs)
    ResolveFrontedElement(mentions, corefs)
    ResolveEntity(mentions, corefs)
    MatchString(mentions, corefs)
    ResolveThirdPersonPronouns(mentions, corefs)
    '''
    
    for sieve in sieve_list:
        sieve(mentions, corefs, stats, wv)

def PlaceCoref(mentions, corefs, ann_file):
    '''
    Writes a coref class with mentions to .ann file object.
    sorted_coref_lists contains the mention identifiers (e.g. T50)
    sorted on TF node. 
    '''
    
    for s in corefs:
        if len(s) > 1:
            ann_file.write(f'*{TAB}Coreference')
            sorted_coref_lists = sorted(s, key=attrgetter('node_tuple'))
            for m in sorted_coref_lists:
                ann_file.write(f' {m}')
            ann_file.write('\n')

def CountCorefClasses(stats, corefs):
    i = 0
    for s in corefs:
        if len(s) > 1:
            i+=1
    stats.coref_classes = i
    
def FindMentionByRPT(c, typ):
    for m in c:
        if m.rpt == typ:
            return m
    
def Identify(c):
    rpt_order = ['PrNP', 'NP', 'PtcP', 'VP', 'PPrP', 'DPrP']
    for typ in rpt_order:
        m = FindMentionByRPT(c, typ)
        if m:
            m.who = m.text
            return
    #c.who = c.first()
            
def AssignWho(corefs):
    for c in corefs:
        if len(c) > 1:
             m = Identify(c)
            
def CorefResolutionStats(stats, filename, coreference_list):

    resolved_corefs = stats.input_corefs - stats.output_corefs
    coref_success_percent = round(stats.coref_success() * 100, 1)
    coref_unresolved_percent = round(stats.coref_unresolved() * 100, 1)
    
    coreference_list.append({'chapter' : filename,
                                 'input corefs' : stats.input_corefs,
                                 'resolved' : resolved_corefs,
                                 'unresolved' : stats.output_corefs,
                                 '%resolved' : coref_success_percent,
                                 '%unresolved' : coref_unresolved_percent,
                                 'classes' : stats.coref_classes
        })
    
    coref_stats_df = pd.DataFrame(coreference_list)
    coref_stats_df = coref_stats_df[['chapter', 'input corefs', 'resolved', 'unresolved', 
                                     '%resolved', '%unresolved', 'classes']]
    
    return coref_stats_df
    
def SumCoResStats(total_stats, my_book_name):
    
    coreference_list = []
    resolved_corefs = total_stats.input_corefs - total_stats.output_corefs
    coref_success_percent = round(total_stats.coref_success() * 100, 1)
    coref_unresolved_percent = round(total_stats.coref_unresolved() * 100, 1)
    
    print('\n',\
        f'Coreference Resolution Statistics {my_book_name}: \n',\
        f'{total_stats.input_corefs} total input corefs \n', \
        f'{resolved_corefs} corefs RESOLVED \n',\
        f'{total_stats.output_corefs} corefs UNRESOLVED \n',\
        f'{coref_success_percent}% corefs RESOLVED \n',\
        f'{coref_unresolved_percent}% corefs UNRESOLVED \n',\
        f'{total_stats.coref_classes} classes')
    
    coreference_list.append({'book' : my_book_name,
                             'input corefs' : total_stats.input_corefs,
                             'resolved' : resolved_corefs,
                             'unresolved' : total_stats.output_corefs,
                             '%resolved' : coref_success_percent,
                             '%unresolved' : coref_unresolved_percent,
                             'classes' : total_stats.coref_classes
        })
    
    coref_total_stats_df = pd.DataFrame(coreference_list)
    coref_total_stats_df = coref_total_stats_df[['book', 'input corefs', 'resolved', 'unresolved', 
                                     '%resolved', '%unresolved', 'classes']]
    
    return coref_total_stats_df

def SieveStats(stats, filename, sieves_list):
    
    resolve_total = stats.resolve_predicate + stats.resolve_1p_2p_pronouns + \
    stats.resolve_vocative + stats.resolve_apposition + stats.resolve_fronted + \
    stats.resolve_entity + stats.match_string + stats.resolve_3p_pronouns
    
    sieves_list.append({'chapter' : filename,
                        '1p 2p pronoun' : stats.resolve_1p_2p_pronouns,
                        'vocative' : stats.resolve_vocative,
                        'string' : stats.match_string,
                        'entity' : stats.resolve_entity,
                        'apposition' : stats.resolve_apposition,
                        'fronted element' : stats.resolve_fronted,
                        'predicate' : stats.resolve_predicate,
                        '3p pronoun' : stats.resolve_3p_pronouns,
                        'total sieves' : resolve_total,
                        'classes' : stats.coref_classes 
        })
        
    sieve_stats_df = pd.DataFrame(sieves_list)
    sieve_stats_df = sieve_stats_df[['chapter', '1p 2p pronoun',
                                     'vocative', 'string', 'entity',
                                     'apposition', 'fronted element',
                                     'predicate', '3p pronoun',
                                     'total sieves', 'classes']]
    return sieve_stats_df

def SumSieveStats(total_stats, book_name):
    '''
    Sum of unite operations in sieves. 
    '''
    sieves_list = []
    
    resolve_total = total_stats.resolve_predicate + total_stats.resolve_1p_2p_pronouns + \
    total_stats.resolve_vocative + total_stats.resolve_apposition + \
    total_stats.resolve_fronted + total_stats.resolve_entity + \
    total_stats.match_string + total_stats.resolve_3p_pronouns
    
    sieves_list.append({'book' : book_name,
                        '1p 2p pronoun' : total_stats.resolve_1p_2p_pronouns,
                        'vocative' : total_stats.resolve_vocative,
                        'string' : total_stats.match_string,
                        'entity' : total_stats.resolve_entity,
                        'apposition' : total_stats.resolve_apposition,
                        'fronted element' : total_stats.resolve_fronted,
                        'predicate' : total_stats.resolve_predicate,
                        '3p pronoun' : total_stats.resolve_3p_pronouns,
                        'total sieves' : resolve_total,
                        'classes' : total_stats.coref_classes 
       })
     
    sieve_total_stats_df = pd.DataFrame(sieves_list)
    sieve_total_stats_df = sieve_total_stats_df[['book', '1p 2p pronoun',
                                                 'vocative', 'string', 'entity',
                                                 'apposition', 'fronted element',
                                                 'predicate', '3p pronoun',
                                                 'total sieves', 'classes']]                  
                       
    print('\n',\
          f'Sieve Statistics {book_name}: \n',\
          f'1p 2p Pronoun Sieve: {total_stats.resolve_1p_2p_pronouns} \n',\
          f'Vocative Sieve: {total_stats.resolve_vocative} \n',\
          f'String Sieve: {total_stats.match_string} \n',\
          f'Entity Sieve: {total_stats.resolve_entity} \n',\
          f'Apposition Sieve: {total_stats.resolve_apposition} \n',\
          f'Fronted Element Sieve: {total_stats.resolve_fronted} \n',\
          f'Predicate Sieve: {total_stats.resolve_predicate} \n',\
          f'3p Pronoun Sieve: {total_stats.resolve_3p_pronouns} \n',\
          f'Total Sieves: {resolve_total} \n',\
          f'Total Classes: {total_stats.coref_classes}'
         )
    
    return sieve_total_stats_df

def PrintMentions(Mentions):
    for m in Mentions:
        print(m.name, m.txttype, m.pargr, m.text, m.person, m.gender, 
              m.number, m.issuffix, m.function, m.rpt, m.node_tuple[0],
              sep='\t'
             )

def PrintCorefClasses(Corefs):
    i = 0 
    for s in Corefs:
        if len(s) > 1:
            i+=1
            print(f'C{i}', sorted(s, key=attrgetter('node_tuple')))
        if len(s) == 1:
            pass

In [14]:
def mimi(my_book_name, first_chapter, last_chapter, wv):
    print('Start.')
    clustername = f'{my_book_name}_{first_chapter:>03}_{last_chapter:>03}'
    global mention_errors 
    mention_errors = OpenErrorFile(clustername)
    total_stats = Statistics()
    coreference_list = []
    sieves_list = []
    i = 0
    coref_lst = []
    for chapter_number in range(first_chapter, last_chapter+1):
        stats = Statistics()
        chn = T.nodeFromSection((my_book_name, chapter_number))
        if chn == None:
            error(f'Chapter not found')
            break
        filename = f'{my_book_name}_{chapter_number:>03}'
        index_dict = ProcessText(chn, filename)
        Mentions = ParseMentions(chn, stats, index_dict)
        ann_file = OpenAnn(filename)
        PlaceMentions(Mentions, ann_file)
        EnrichMentions(Mentions)
        Corefs = MakeCorefSets(Mentions)
        stats.input_corefs = len(Corefs)
        sieve_list = MakeSieveList()
        ExecuteSieves(sieve_list, Mentions, Corefs, stats, wv)
        stats.output_corefs = len(Corefs)
        PlaceCoref(Mentions, Corefs, ann_file)
        CloseFile(ann_file)
        AssignWho(Corefs)
        
        # statistics
        CountCorefClasses(stats, Corefs)
        sieve_stats_df = SieveStats(stats, filename, sieves_list)
        coref_stats_df = CorefResolutionStats(stats, filename, coreference_list)
        total_stats += stats
        coref_lst.append(Corefs)
        #CheckList(Corefs)
    mention_stats_df = MentionParseStats(total_stats, my_book_name)
    #print('Parsing and resolution: done.')
    CloseFile(mention_errors)
    
    ##PrintMentions(Mentions) #For GoMiMi()
    ##CheckList(Corefs) #For GoMiMi()
    
    #print(rule_count)
    
    coref_total_df = SumCoResStats(total_stats, my_book_name)
    sieve_total_df = SumSieveStats(total_stats, my_book_name)
    
    return Mentions, coref_lst, mention_stats_df, coref_stats_df, \
            sieve_stats_df, coref_total_df, sieve_total_df

In [15]:
mentions, corefs, mention_stats_df, coref_stats_df,\
        sieve_stats_df, coref_total_df, sieve_total_df = mimi('Psalms', 1, 150, 1) #105, 106, 136 (veel herhaling)

Start.

 Mention Parse Statistics Psalms: 
 18599 phrase atoms INPUT 
 18552 phrase atoms SUCCESFULLY parsed 
 +27 extra mentions SUCCESFULLY parsed from phrase atoms 
 -98 phrase atoms without mentions 
 47 phrase atom parse ERRORS 
 +0 extra mentions SUCCESFULLY parsed from phrase atom errors 
 -44 phrase atoms without mentions from phrase atom ERRORS 
 18484 mention coreference input 
 99.7% parsing succes 
 0.3% parsing error
Parsing and resolution: done.

 Coreference Resolution Statistics Psalms: 
 18484 total input corefs 
 10554 corefs RESOLVED 
 7930 corefs UNRESOLVED 
 57.1% corefs RESOLVED 
 42.9% corefs UNRESOLVED 
 2647 classes

 Sieve Statistics Psalms: 
 1p 2p Pronoun Sieve: 2756 
 Vocative Sieve: 539 
 String Sieve: 1757 
 Entity Sieve: 336 
 Apposition Sieve: 50 
 Fronted Element Sieve: 20 
 Predicate Sieve: 3130 
 3p Pronoun Sieve: 1966 
 Total Sieves: 10554 
 Total Classes: 2647


In [ ]:
def main(argv):
    try:
        opts, args = getopt.getopt(argv, 'v', [])
    except getopt.GetoptError:
        Usage()
    print(len(args), len(argv))
    if len(args) == 2:
        last_chapter = int(args[1])
    elif len(args) == 3:
        last_chapter = int(args[2])
    else:
        Usage()   
    
    first_chapter = int(args[1])
    book_name = args[0]
    
    mimi(book_name, first_chapter, last_chapter)

if __name__ == "__main__":
    main(argv[1:])